# Features

data_stocks.tickers['ADS.DE'].calendar.index.rename('date', inplace=True)

weekday diff_days 
stock = stock.reset_index(level=0)
stock['Daily Change'] = stock['Adj. Close'] - stock['Adj. Open']

n_shares = math.floor(money / start_price)
end_price = (price_high + start_price + end_price)
profit = n_shares * (end_price - start_price)

primary key -> remove duplicates (same dates)

* data_stocks
    * tickers:
        * name e.g. ADS.DE
            * calendar
                * Earnings Date -> date
                * earnings_average
                * earnings_low
                * earnings_high
                * revenue_average
                * revenue_low
                * revenue_high
            * recommendations
                * Date -> date
                * firm: Berenberg, Exane BNP Paribas
                * to_grade: Buy, Neutral
                * from_grade: Hold, ''
                * action: up, init
            * info
                * 52WeekChange
                * SandP52WeekChange
                * averageDailyVolume10Day
                * averageVolume
                * averageVolume10days
                * beta
                * ask
                * askSize
                * bid
                * bidSize
                * bookValue
                * dayHigh
                * dayLow
                * dividendRate
                * dividendYield
                * earningsQuarterlyGrowth
                * enterpriseToEbitda
                * enterpriseToRevenue
                * enterpriseValue
                * exDividendDate
                * fiftyDayAverage
                * fiftyTwoWeekHigh
                * fiftyTwoWeekLow
                * fiveYearAvgDividendYield
                * floatShares
                * forwardEps
                * forwardPE
                * fullTimeEmployees
                * heldPercentInsiders
                * heldPercentInstitutions
                * marketCap
                * netIncomeToCommon
                * open
                * payoutRatio
                * pegRatio
                * previousClose
                * priceToBook
                * profitMargins
                * sector: Consumer Cyclical
                * sharedOutstanding
                * symbol
                * regularMarketDayHigh
                * regularMarketDayLow
                * regularMarketOpen
                * regularMarketPreviousClose
                * regularMarketVolume
                * trailingAnnualDividendRate
                * trailingAnnualDividendYield
                * trailingEps
                * trailingPE
                * volume               
            * sustainability
                * Date -> date
                * palmoil: t/f
                * controversialweapons: t/f
                * gambling: t/f
                * socialscore: float
                * nuclear: t/f
                * furleather: t/f
                * alcoholic: t/f
                * gmo: t/f
                * catholic: t/f
                * socialpercentile: float
                * peercount: int
                * relatedcontroversy: text
                * governancescore: float
                * environmentpercentile: int
                * animaltesting: t/f
                * tobacco: t/f
                * totalesg: float
                * highestcontroversy: int
                * esgperformance: OUT_PERF
                * coal: t/f
                * pesticides: t/f
                * adult: t/f
                * percentile: float
                * peergroup: Textiles & Apparel
                * smallarms: t/f
                * environmentscore: float    
                * governancepercentile: float
                * militarycontract: t/f
            * earnings
                * Quarter -> date
                * revenue
                * earnings
            * financials (quarter)
                * "" -> date
                * research_development: None
                * effect_of_accounting_charges: None
                * income_before_tax: float
                * minority_interest: float (neg)
                * net_income: float
                * selling_general_administrative: float
                * gross_profit: float
                * ebit: float
                * operating_income: float
                * other_operating_expenses: float (neg)
                * interest_expense: float (neg)
                * extraordinary_items: None
                * non_recurring: None
                * other_items: None
                * income_tax_expense: float
                * total_revenue: float
                * total_operating_expenses: float
                * cost_of_revenue: float
                * total_other_income_expense_net: float (neg)
                * discontinued_operations: float
                * net_income_from_continuing_ops: float
                * net_income_applicable_to_common_shares: float
            * balancesheet (quarter)
                * "" -> date
                * intangible_assets: float
                * total_liab: float
                * total_stockholder_equity: float
                * minority_interest
                * other_current_liab
                * total_assets
                * common_stock
                * other_current_assets
                * retained_earnings
                * other_liab
                * good_will
                * treasury_stock
                * other_assets
                * cash
                * total_current_liabilities
                * deferred_long_term_asset_charges
                * short_long_term_debt
                * other_stockholder_equity
                * property_plant_equipment
                * total_current_assets
                * long_term_investments
                * net_tangible_assets
                * short_term_investments
                * net_receivables
                * long_term_debt
                * inventory
                * accounts_payable
            * cashflow (quarter)
                * investments
                * change_to_liabilities
                * total_cashflows_from_investing_activities
                * net_borrowings
                * total_cash_from_financing_activities
                * issuance_of_stock
                * net_income
                * change_in_cash
                * repurchase_of_stock
                * effect_of_exchange_rate
                * total_cash_from_operating_activities
                * depreciation
                * other_cashflows_from_investing_activities
                * change_to_inventory
                * change_to_account_receivables
                * other_cashflows_from_financing_activities
                * change_to_netincome
                * capital_expenditures
                * dividends_paid
            * dividends: EMPTY
            * splits: EMPTY
            * history
                * Date -> date
                * open
                * high
                * low
                * close
                * dividends
                * stock_splits
            * calls: EMPTY
            * puts: EMPTY
    * downloads:
        * name e.g. ADS.DE
            * close
                * Date -> date
                * close
            * high
                * Date
                * high
            * low
                * Date
                * low
            * open
                * Date
                * open
            * volume
                * Date
                * volume
    
* benchmark .. * 90
    * tickers: 
        * same as stocks
    * downloads:
        * name e.g. ADS.DE
            * close
                * Date -> date
                * close
            * high
                * Date
                * high
            * low
                * Date
                * low
            * open
                * Date
                * open
            * volume
                * Date
                * volume